In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
import numpy as np

import scipy.special
import glob
import os

from scipy.stats import gaussian_kde

from IPython.core.display import display, HTML
from IPython.display import clear_output, display

from bokeh.models import LinearColorMapper, BasicTicker, PrintfTickFormatter, ColorBar, ColumnDataSource, Button
from bokeh.models.widgets import Select
from bokeh.models.callbacks import CustomJS
from bokeh.models.glyphs import HBar
from bokeh.models.widgets import Panel, Tabs

from bokeh.plotting import figure
from bokeh.plotting import output_notebook

from bokeh.layouts import row, column
from bokeh.layouts import gridplot

from bokeh.io import show, push_notebook

output_notebook()

# If you want to widen the page
# you can modify *width* to the one you prefer

display(HTML("<style>.container { width:80% !important; }</style>"))

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive


def download_sheets(_):
    try:
        gauth = GoogleAuth()
        gauth.LocalWebserverAuth()

        drive = GoogleDrive(gauth)

        file_list = drive.ListFile(
            {'q': "mimeType='application/vnd.google-apps.spreadsheet' and sharedWithMe and 'cboullay@gmail.com' in writers and (title contains 'TP-NOTE' or title contains 'CAML' or title contains 'C#')"}).GetList()
        for file1 in file_list:
            file1.GetContentFile(
                'sheets/' + file1['title'] + '.xlsx', mimetype='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')
            print('Downloaded {}'.format(file1['title']))

        print('Done')
    except:
        print("You're missing the client_secrets.json file, go check the README")


def on_button_click(_):
    print('Hello')


button = Button(label='Download sheets')

button.on_click(download_sheets)

show(button)

In [ ]:
# Get files
files = [f for f in glob.glob(os.getcwd() + "/sheets/*.xlsx")]
raw_files = [f.split('/')[-1].split('\\')[-1] for f in files]


def read_practical(file):
    # Practical selection

    TP = pd.ExcelFile(file)

    # Grade Processing

    df = dict()
    for sheet in TP.sheet_names[1:]:
        df[sheet] = TP.parse(sheet, header=None)

    # Getting all the different exercises
    cols = []

    for i in range(len(df['promo'].loc[0])):
        # append second line if first line is nan
        exercise = str(
            df['promo'].loc[int(str(df['promo'].loc[0][i]) == 'nan')][i])
        while (exercise in cols and str(exercise) != 'nan'):
            exercise += "_"
        cols.append(exercise)

    # Grouping columns two by two, so that each column represent one exercise
    for i in range(len(cols)):
        if str(cols[i]) == 'nan':
            cols[i] = cols[i-1] + "_to_drop"

    cols[0] = 'login'  # Just in case someone forgets...

    promo = pd.DataFrame()

    # Data cleaning for every sheet in the .xlsx
    for sheet in TP.sheet_names[2 - ('CTRL' in file or 'PART' in file):]:
        df[sheet].columns = cols
        df[sheet].drop([0, 1, 2], inplace=True)
        df[sheet][cols[4:-2]] = df[sheet][cols[4:-2]
                                          ].apply(lambda a: pd.to_numeric(a, errors='coerce'))
        df[sheet].set_index('login', inplace=True)
        df[sheet]['corrector'].fillna(method='ffill', inplace=True)
        df[sheet].fillna(0, inplace=True)
        for i in range(4, len(cols) - 3, 2):
            df[sheet][cols[i]] += df[sheet][cols[i+1]]
            # Check typo in grades
            df[sheet][cols[i]].apply(lambda a: a if a <= 2 else 2)
        df[sheet].drop(cols[5:-2:2], axis=1, inplace=True)
        df[sheet]['total'] = df[sheet][cols[4:-2:2]
                                       ].sum(axis=1) / (len(cols) - 6) * 100
        promo = pd.concat([promo, df[sheet]])
    return promo


practicals = [read_practical(f) for f in files]

In [ ]:
# Normalization

stats_promo = dict()

for sheet in raw_files:
    sheet_stats_promo = practicals[raw_files.index(sheet)].groupby(['gr']).mean()
    sheet_stats_promo.loc[:, sheet_stats_promo.columns !=
                    'total'] *= 50  # Everything is in % now
    sheet_stats_promo = sheet_stats_promo.round(2)
    stats_promo[sheet] = sheet_stats_promo

In [ ]:
colors = ['#827E9A', '#E69999', '#F8BD7F', '#3E4E50', '#B388EB',
          '#8093F1', '#C4B7CB', '#BBC7CE', '#98E2C6', '#99B2DD', '#EDBFC6']


def make_practical_kde_plots(exercices):
    plots = []
    yr = [0, 2.3]

    for i, ex in enumerate(exercices):
        xs = []
        ys = []
        subset = exercices[ex].values

        kde = gaussian_kde(subset)
        # Evenly space x values
        x = np.linspace(0, 2, 100)
        # Evaluate pdf at every value of x
        y = kde.pdf(x)

        # Append the values to plot
        xs.append(list(x))
        ys.append(list(y))

        kernel_source = ColumnDataSource(data={'x': xs, 'y': ys})
        p = figure(width=400, height=100, tools="", y_range=yr)
        p.multi_line('x', 'y', color=colors[i % len(
            colors)], legend=ex, line_width=3, source=kernel_source)
        p.legend.location = 'top_left'
        plots.append(p)

    return gridplot(plots, ncols=2 + (len(plots) > 6))

def make_heatmap_practical(sheet_stats_promo):
    
    data = {'exo': [], 'group': [], 'total': []}

    for index, cols in sheet_stats_promo.iterrows():
        for practical in sheet_stats_promo.columns:
            data['exo'].append(practical)
            data['group'].append(index)
            data['total'].append(sheet_stats_promo[practical][index])

    tp_stats = pd.DataFrame(data=data)
    practical = raw_files[0]
    practical = practical.split('.')[0]

    colors = ["#ea9999", "#f4c7c3", "#fce8b2", "#b7e1cd", "#57bb8a"]
    mapper = LinearColorMapper(
        palette=colors, low=tp_stats.total.min(), high=tp_stats.total.max())

    p = figure(title="Class averages for practical: {0}".format(select_sheet.value),
               x_range=sheet_stats_promo.columns.tolist(), y_range=[index for index, cols in sheet_stats_promo.iterrows()][::-1],
               x_axis_location="above", plot_height=600,
               tools="", toolbar_location='below', width_policy='max',
               tooltips=[('group', '@group'), ('exercise', '@exo'), ('mean', '@total')])

    p.axis.axis_line_color = None
    p.axis.major_tick_line_color = None
    p.axis.major_label_text_font_size = "10pt"
    p.axis.major_label_standoff = 0
    p.axis.major_label_orientation = 'vertical'

    p.rect(x="exo", y="group", width=1, height=1,
           source=tp_stats,
           fill_color={'field': 'total', 'transform': mapper},
           line_color='white')

    color_bar = ColorBar(color_mapper=mapper, major_label_text_font_size="8pt",
                         ticker=BasicTicker(desired_num_ticks=len(colors)),
                         formatter=PrintfTickFormatter(format="%d%%"),
                         label_standoff=8, border_line_color=None, location=(0, 0))
    p.add_layout(color_bar, 'right')

    return p

In [ ]:
def group_change(attr, old, new):
    select_student.options = practicals[0][practicals[0]
                                           ['gr'] == new].index.tolist()
    select_group.value = new


def student_change(attr, old, new):
    select_student.value = new


def sheet_change(attr, old, new):
    select_sheet.value = new
    p = practicals[raw_files.index(new)]
    exercices = p[p.columns[4: -3]]
    
    c = column(make_heatmap_practical(stats_promo[new]), make_practical_kde_plots(exercices))
    tab1 = Panel(child=c, title='Practical info')
    tabs.tabs = [tab1, tab2, tab3]
    push_notebook(handle=t)


select_sheet = Select(title='Sheets', options=raw_files, value=raw_files[0])
select_group = Select(
    title='Group', options=practicals[0]['gr'].unique().tolist())
select_group.value = select_group.options[0]
select_student = Select(
    title='Student', options=practicals[0][practicals[0]['gr'] == select_group.value].index.tolist())
select_student.value = select_student.options[0]

select_group.on_change('value', group_change)
select_sheet.on_change('value', sheet_change)
select_student.on_change('value', student_change)


def plaf(doc):
    doc.add_root(row(select_group, select_student, select_sheet))


show(plaf)

p = practicals[raw_files.index(select_sheet.value)]
exercices = p[p.columns[4: -3]]

c = column(make_heatmap_practical(stats_promo[select_sheet.value]), make_practical_kde_plots(exercices))

tab1 = Panel(child=c, title='Practical info')

p2 = figure(plot_width=300, plot_height=300)
p2.line([1, 2, 3, 4, 5], [6, 7, 2, 4, 5],
        line_width=3, color="navy", alpha=0.5)
tab2 = Panel(child=p2, title='Class info')

tab3 = Panel(child=p2, title='Student info')

tabs = Tabs(tabs=[tab1, tab2, tab3])

t = show(tabs, notebook_handle=True)

In [ ]:
student_grades = pd.DataFrame([practicals[i].loc[select_student.value, ['total']].append(
    pd.Series([raw_files[i].split('.')[0]], ['TP'])) for i in range(len(raw_files))]).round(2)

s_grades = student_grades
s_grades.index = s_grades['TP'].to_list()
s_grades.drop(['TP'], axis=1, inplace=True)

s_grades

In [ ]:
student_grades = pd.DataFrame([practicals[i].loc[select_student.value, ['total']].append(
    pd.Series([raw_files[i].split('.')[0]], ['TP'])) for i in range(len(raw_files))])

student_grades['color'] = student_grades.total.apply(
    lambda grade: 'blue' if grade > 95 else 'red')

source = ColumnDataSource(student_grades)
averages = [practicals[raw_files.index(i)].mean()['total'] for i in raw_files]

p = figure(y_range=student_grades['TP'], tools='')
glyph = HBar(y="TP", right="total", left=0, height=0.5, fill_color="#b3de69")
p.add_glyph(source, glyph)

show(p)

In [ ]:
show(make_heatmap_practical(stats_promo))